In [2]:
# import libraries
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
# %matplotlib inline

from numpy.random import seed
# from tensorflow import set_random_seed
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)


from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
import os
# os.chdir('C:/Lehigh/Rick- Anomaly Detection/Code')

In [22]:
df = pd.read_csv("/content/drive/MyDrive/CNN-KF/Clean_data.csv")

df = df.rename({'InVehicle_Longitudinal_Speed':'inveh_long_spd', 'GPS_Speed': 'gps_speed', 'InVehicle_Longitudinal_Accel': 'inveh_long_acc'}, axis = 1)

df = df.dropna()
df.shape

(30000, 4)

In [5]:
simulated_df = pd.read_csv("/content/drive/MyDrive/CNN-KF/Instant attack/25/Sensor3_values_instant_25.csv", index_col = 'Unnamed: 0')
simulated_gt = pd.read_csv("/content/drive/MyDrive/CNN-KF/Instant attack/25/Ground_truth_test_set_instant_25_sensor3.csv")

In [6]:
# Assuming last 5958 rows of Sensor3_values_instant_25.csv belongs to test samples

simulated_df_test = simulated_df.iloc[-simulated_gt.shape[0]:]
simulated_df_test = simulated_df_test.rename({'InVehicle_Longitudinal_Speed':'inveh_long_spd', 'GPS_Speed': 'gps_speed', 'InVehicle_Longitudinal_Accel': 'inveh_long_acc'}, axis = 1)

In [7]:
train_size = 24041
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], simulated_df_test
print(train.shape, test.shape)

(24041, 3) (5958, 3)


In [20]:
def time_segments_aggregate(X, interval, time_column, method=['mean']):
    """Aggregate values over given time span.
    Args:
        X (ndarray or pandas.DataFrame):
            N-dimensional sequence of values.
        interval (int):
            Integer denoting time span to compute aggregation of.
        time_column (int):
            Column of X that contains time values.
        method (str or list):
            Optional. String describing aggregation method or list of strings describing multiple
            aggregation methods. If not given, `mean` is used.
    Returns:
        ndarray, ndarray:
            * Sequence of aggregated values, one column for each aggregation method.
            * Sequence of index values (first index of each aggregated segment).
    """
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)

    X = X.sort_values(time_column).set_index(time_column)

    if isinstance(method, str):
        method = [method]

    start_ts = X.index.values[0]
    max_ts = X.index.values[-1]

    values = list()
    index = list()
    while start_ts <= max_ts:
        end_ts = start_ts + interval
        subset = X.loc[start_ts:end_ts - 1]
        aggregated = [
            getattr(subset, agg)(skipna=True).values
            for agg in method
        ]
        values.append(np.concatenate(aggregated))
        index.append(start_ts)
        start_ts = end_ts

    return np.asarray(values), np.asarray(index)

In [34]:
train, index = time_segments_aggregate(df, interval=5, time_column='Epoch')

In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train)
X_train = scaler.transform(train)
# X_test = scaler.transform(test)

In [29]:
def rolling_window_sequences(X, index, window_size, target_size, step_size, target_column,
                             drop=None, drop_windows=False):
    """Create rolling window sequences out of time series data.
    The function creates an array of input sequences and an array of target sequences by rolling
    over the input sequence with a specified window.
    Optionally, certain values can be dropped from the sequences.
    Args:
        X (ndarray):
            N-dimensional sequence to iterate over.
        index (ndarray):
            Array containing the index values of X.
        window_size (int):
            Length of the input sequences.
        target_size (int):
            Length of the target sequences.
        step_size (int):
            Indicating the number of steps to move the window forward each round.
        target_column (int):
            Indicating which column of X is the target.
        drop (ndarray or None or str or float or bool):
            Optional. Array of boolean values indicating which values of X are invalid, or value
            indicating which value should be dropped. If not given, `None` is used.
        drop_windows (bool):
            Optional. Indicates whether the dropping functionality should be enabled. If not
            given, `False` is used.
    Returns:
        ndarray, ndarray, ndarray, ndarray:
            * input sequences.
            * target sequences.
            * first index value of each input sequence.
            * first index value of each target sequence.
    """
    out_X = list()
    out_y = list()
    X_index = list()
    y_index = list()
    target = X[:, target_column]

    if drop_windows:
        if hasattr(drop, '__len__') and (not isinstance(drop, str)):
            if len(drop) != len(X):
                raise Exception('Arrays `drop` and `X` must be of the same length.')
        else:
            if isinstance(drop, float) and np.isnan(drop):
                drop = np.isnan(X)
            else:
                drop = X == drop

    start = 0
    max_start = len(X) - window_size - target_size + 1
    while start < max_start:
        end = start + window_size

        if drop_windows:
            drop_window = drop[start:end + target_size]
            to_drop = np.where(drop_window)[0]
            if to_drop.size:
                start += to_drop[-1] + 1
                continue

        out_X.append(X[start:end])
        out_y.append(target[end:end + target_size])
        X_index.append(index[start])
        y_index.append(index[end])
        start = start + step_size

    return np.asarray(out_X), np.asarray(out_y), np.asarray(X_index), np.asarray(y_index)

In [40]:
X, y, X_index, y_index = rolling_window_sequences(X_train, index, 
                                                  window_size=100, 
                                                  target_size=1, 
                                                  step_size=1,
                                                  target_column=0)

In [43]:
y.shape

(5900, 1)

In [44]:
# Check gpu envrionmental 
import tensorflow as tf
import logging
import math

gpus = tf.config.experimental.list_physical_devices('GPU') 
if gpus: 
    try: 
        for gpu in gpus: 
            tf.config.experimental.set_memory_growth(gpu, True) 
    except RuntimeError as e: 
        print(e)
print (gpus)

[]


In [45]:
LOGGER = logging.getLogger(__name__)

In [46]:
pip install similaritymeasures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import numpy as np
import pandas as pd
import pickle

#import tensorflow as tf
import keras
import matplotlib.pyplot as plt

import similaritymeasures as sm
from tensorflow.keras import backend as K

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Bidirectional, LSTM, Flatten, Dense, Reshape, UpSampling1D, TimeDistributed
from tensorflow.keras.layers import Activation, Conv1D, LeakyReLU, Dropout, Add, Layer
from tensorflow.compat.v1.keras.layers import CuDNNLSTM as CUDNNLSTM
from tensorflow.keras.optimizers import Adam

from functools import partial
from scipy import integrate, stats

In [48]:
class RandomWeightedAverage(Layer):
    def _merge_function(self, inputs):
        alpha = K.random_uniform((64, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

In [49]:
def build_encoder_layer(input_shape, encoder_reshape_shape):    
    
    input_layer = layers.Input(shape=input_shape)
    
    x = layers.Bidirectional(CUDNNLSTM(units=100, return_sequences=True))(input_layer)
    x = layers.Flatten()(x)
    x = layers.Dense(20)(x)
    x = layers.Reshape(target_shape=encoder_reshape_shape)(x)
    model = keras.models.Model(input_layer, x, name='encoder')
    
    return model

def build_generator_layer(input_shape, generator_reshape_shape):
    
    input_layer = layers.Input(shape=input_shape)
    
    x = layers.Flatten()(input_layer)
    x = layers.Dense(generator_reshape_shape[0])(x)
    x = layers.Reshape(target_shape=generator_reshape_shape)(x)
    x = layers.Bidirectional(CUDNNLSTM(units=64, return_sequences=True), merge_mode='concat')(x)
    x = layers.UpSampling1D(size=2)(x)
    x = layers.Bidirectional(CUDNNLSTM(units=64, return_sequences=True), merge_mode='concat')(x)
    x = layers.TimeDistributed(layers.Dense(1))(x)
    x = layers.Activation(activation='tanh')(x)
    model = keras.models.Model(input_layer, x, name='generator')
    
    return model
    

def build_critic_x_layer(input_shape):
    
    input_layer = layers.Input(shape=input_shape)
    
    x = layers.Conv1D(filters=64, kernel_size=5)(input_layer)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Conv1D(filters=64, kernel_size=5)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Conv1D(filters=64, kernel_size=5)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Conv1D(filters=64, kernel_size=5)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.25)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=1)(x)
    model = keras.models.Model(input_layer, x, name='critic_x')
    
    return model 


def build_critic_z_layer(input_shape):
    
    input_layer = layers.Input(shape=input_shape)
    
    x = layers.Flatten()(input_layer)
    x = layers.Dense(units=100)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.2)(x)    
    x = layers.Dense(units=100)(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Dropout(rate=0.2)(x)  
    x = layers.Dense(units=1)(x)
    model = keras.models.Model(input_layer, x, name='critic_z')
    
    return model

In [50]:
def wasserstein_loss(y_true, y_pred):
#    return tf.reduce_mean(y_true * y_pred)
    return K.mean(y_true * y_pred)


In [55]:
window_size = 200

In [56]:
# Layer Parameters
encoder_input_shape = (window_size, 1)
generator_input_shape = (20, 1)

critic_x_input_shape = (window_size, 1)
critic_z_input_shape = (20,1)

encoder_reshape_shape = (20, 1)
generator_reshape_shape = (window_size//2, 1) # window_size//3 <- 3 is Upsampling size

learning_rate = 0.0005

# Build Model
encoder = build_encoder_layer(input_shape=encoder_input_shape,
                              encoder_reshape_shape=encoder_reshape_shape)

generator = build_generator_layer(input_shape=generator_input_shape,
                                  generator_reshape_shape=generator_reshape_shape)

critic_x = build_critic_x_layer(input_shape=critic_x_input_shape)
critic_z = build_critic_z_layer(input_shape=critic_z_input_shape)

encoder_optimizer = tf.keras.optimizers.Adam(learning_rate)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate)
critic_x_optimizer = tf.keras.optimizers.Adam(learning_rate)
critic_z_optimizer = tf.keras.optimizers.Adam(learning_rate)

In [57]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 200, 1)]          0         
                                                                 
 bidirectional_6 (Bidirectio  (None, 200, 200)         82400     
 nal)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 40000)             0         
                                                                 
 dense_10 (Dense)            (None, 20)                800020    
                                                                 
 reshape_4 (Reshape)         (None, 20, 1)             0         
                                                                 
Total params: 882,420
Trainable params: 882,420
Non-trainable params: 0
_____________________________________________________

In [58]:
generator.summary() 

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 20, 1)]           0         
                                                                 
 flatten_7 (Flatten)         (None, 20)                0         
                                                                 
 dense_11 (Dense)            (None, 100)               2100      
                                                                 
 reshape_5 (Reshape)         (None, 100, 1)            0         
                                                                 
 bidirectional_7 (Bidirectio  (None, 100, 128)         34304     
 nal)                                                            
                                                                 
 up_sampling1d_2 (UpSampling  (None, 200, 128)         0         
 1D)                                                     

In [59]:
critic_x.summary()

Model: "critic_x"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 200, 1)]          0         
                                                                 
 conv1d_6 (Conv1D)           (None, 196, 64)           384       
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 196, 64)           0         
                                                                 
 dropout_7 (Dropout)         (None, 196, 64)           0         
                                                                 
 conv1d_7 (Conv1D)           (None, 192, 64)           20544     
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 192, 64)           0         
                                                                 
 dropout_8 (Dropout)         (None, 192, 64)           0  

In [60]:
critic_z.summary()

Model: "critic_z"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 20, 1)]           0         
                                                                 
 flatten_9 (Flatten)         (None, 20)                0         
                                                                 
 dense_14 (Dense)            (None, 100)               2100      
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 100)               0         
                                                                 
 dropout_11 (Dropout)        (None, 100)               0         
                                                                 
 dense_15 (Dense)            (None, 100)               10100     
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 100)               0  

In [61]:
latent_dim = 20
shape = (window_size, 1)

In [62]:
@tf.function
def critic_x_train_on_batch(x, z):
    # Loss 크게 이상 없음 
    with tf.GradientTape() as tape:
        
        valid_x = critic_x(x)
        x_ = generator(z)
        fake_x = critic_x(x_)
        
        # Interpolated 
        alpha = tf.random.uniform([batch_size, 1, 1], 0.0, 1.0)
        interpolated = alpha * x + (1 - alpha) * x_ 
        
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = critic_x(interpolated)
        
        grads = gp_tape.gradient(pred, interpolated)
        grad_norm = tf.norm(tf.reshape(grads, (batch_size, -1)), axis=1)
        gp_loss = 10.0*tf.reduce_mean(tf.square(grad_norm - 1.))
#         grads = tf.square(grads)
#         ddx = tf.sqrt(tf.reduce_sum(grads, axis=np.arange(1, len(grads.shape))))
#        gp_loss = tf.reduce_mean((1.0 - ddx) ** 2)
                
        loss1 = wasserstein_loss(-tf.ones_like(valid_x), valid_x)
        loss2 = wasserstein_loss(tf.ones_like(fake_x), fake_x)
        #loss = tf.add_n([loss1, loss2, gp_loss*10.0])        
        loss = loss1 + loss2 + gp_loss
#        loss = tf.reduce_mean(loss)
                        
    gradients = tape.gradient(loss, critic_x.trainable_weights)
    critic_x_optimizer.apply_gradients(zip(gradients, critic_x.trainable_weights))
    return loss

In [63]:
@tf.function
def critic_z_train_on_batch(x, z):
    
    with tf.GradientTape() as tape:

        z_ = encoder(x)   
        valid_z = critic_z(z)             
        fake_z = critic_z(z_) # <- critic_z 의 결과가 매우 않음 
        
        # Interpolated 
        alpha = tf.random.uniform([batch_size, 1, 1], 0.0, 1.0)
        interpolated = alpha * z + (1 - alpha) * z_ 
                
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = critic_z(interpolated, training=True)
            
        grads = gp_tape.gradient(pred, interpolated)
        grad_norm = tf.norm(tf.reshape(grads, (batch_size, -1)), axis=1)
        gp_loss = 10.0*tf.reduce_mean(tf.square(grad_norm - 1.))

#         grads = tf.square(grads)
#         ddx = tf.sqrt(tf.reduce_sum(grads, axis=np.arange(1, len(grads.shape))))
#         gp_loss = tf.reduce_mean((1.0 - ddx) ** 2)
        
        loss1 = wasserstein_loss(-tf.ones_like(valid_z), valid_z)
        loss2 = wasserstein_loss(tf.ones_like(fake_z), fake_z) # <- 이게 미친듯이 뜀. 
        loss = loss1 + loss2 + gp_loss
#        loss = tf.reduce_mean(loss)
        
    gradients = tape.gradient(loss, critic_z.trainable_weights)
    critic_z_optimizer.apply_gradients(zip(gradients, critic_z.trainable_weights))
    return loss

In [64]:
@tf.function
def enc_gen_train_on_batch(x, z):
    with tf.GradientTape() as enc_tape:
        
        z_gen_ = encoder(x, training=True)
        x_gen_ = generator(z, training=False)        
        x_gen_rec = generator(z_gen_, training=False)
        
        fake_gen_x = critic_x(x_gen_, training=False)
        fake_gen_z = critic_z(z_gen_, training=False)
        
        loss1 = wasserstein_loss(fake_gen_x, -tf.ones_like(fake_gen_x))
        loss2 = wasserstein_loss(fake_gen_z, -tf.ones_like(fake_gen_z))
        loss3 = 10.0*tf.reduce_mean(tf.keras.losses.MSE(x, x_gen_rec))

        enc_loss = loss1 + loss2 + loss3
        
    gradients_encoder = enc_tape.gradient(enc_loss, encoder.trainable_weights)
    encoder_optimizer.apply_gradients(zip(gradients_encoder, encoder.trainable_weights))

    with tf.GradientTape() as gen_tape:
        
        z_gen_ = encoder(x, training=False)
        x_gen_ = generator(z, training=True)        
        x_gen_rec = generator(z_gen_, training=True)
        
        fake_gen_x = critic_x(x_gen_, training=False)
        fake_gen_z = critic_z(z_gen_, training=False)
        
        loss1 = wasserstein_loss(fake_gen_x, -tf.ones_like(fake_gen_x))
        loss2 = wasserstein_loss(fake_gen_z, -tf.ones_like(fake_gen_z))
        loss3 = 10.0*tf.reduce_mean(tf.keras.losses.MSE(x, x_gen_rec))

        gen_loss = loss1 + loss2 + loss3
        
    gradients_generator = gen_tape.gradient(gen_loss, generator.trainable_weights)    
    generator_optimizer.apply_gradients(zip(gradients_generator, generator.trainable_weights))    
    return enc_loss, gen_loss

In [65]:
# Train parameters
batch_size = 32
n_critics = 5
epochs = 1000

In [66]:
X.shape


(5900, 100, 3)

In [67]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [69]:
# Data Reshape
X_copy = X.reshape((-1, shape[0], 1))
# X_ = np.copy(X)
X_copy.shape

(8850, 200, 1)